In [38]:
#Instalar los pip install antes de correr el programa para evitar errores 

#pip install --user opencv-contrib-python    Instala en consola
#pip install --upgrade pip              Instala en consola
#captura imagenes 

#Importante, instalar la contribucion python ya que por defecto se instala es pip install opencv phyton
#por lo cual esta genera error ya que la contribucion face no esta. 

import cv2

web_cam = cv2.VideoCapture(0)    #en caso de no tener camara web el programa al no reconocerla genera- 
#error, importante tener camara web antes compilar esta parte, ya que es necesaria para las fotografias

cascPath = "./Cascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

count = 0

while(True):   
    _, imagen_marco = web_cam.read()

    grises = cv2.cvtColor(imagen_marco, cv2.COLOR_BGR2GRAY)

    rostro = faceCascade.detectMultiScale(grises, 1.5, 5)

    for(x,y,w,h) in rostro:
        cv2.rectangle(imagen_marco, (x,y), (x+w, y+h), (255,0,0), 4)
        count += 1
       
        cv2.imwrite("./images/cristiano/cristiano"+str(count)+".jpg", grises[y:y+h, x:x+w]) #esta parte
        #es importante  es la ruta donde se guadan las capturas de pantalla dentro de images crear la 
        #carpeta con el nombre de la persona que se va a registar ejemplo ./images/nombrepersona/nombre
        #con el que quiera que aparezcan las fotos. 
        
       
        cv2.imshow("Creando Dataset", imagen_marco)

    if cv2.waitKey(1) & 0xFF == ord('q'):  #esta parte es para si queremos salir de la ventana emergente antes de 
        #que se cumplan las capturas 
        break
    
    elif count >= 400:  #numero de fotos que necesita para salise del while.Nota: entre mas fotos mas facil se hace el reconocimiento 
        break


# Cuando todo está hecho, liberamos la captura
web_cam.release()
cv2.destroyAllWindows()

# Al leer todas las recomendaciones le damos a correr, algo muy importante es mirar que la camara enciende el led
#pero no dará imagen hasta que detecte que hay un rostro frente a ella, por lo que si no aparece
# moverse frente ala web cam tratando de que esta detecte que se esta enfrente un rostro


#importante EN CASO QUE ACABE LA CAPTURA DE FOTOS Y VEAS QUE NO SE APAGA EL LED DE LA WEB CAM LE DAS A 
#"RESTAR & CLEAR KERNEL" PARA QUE SE APAGUE. NO SIGAS SI LA VES ENCENDIDA POR QUE TE GENERA ERROR CUANDO LE DAS A CORRER 
#EN LA ETAPA DE RECONOCIMIENTO. 
#ESTA PARTE DE REINICIAR EL KERNEL SOLO EN ESTE CUADRO NO AFECTA POR QUE YA TOMO LAS FOTOS Y LES ASIGNO NOMBRE

In [ ]:
#entrenamiento

In [39]:
#ESTA es la parte de entrenamiento por lo general esta parte crea un archivo de entrenamiendo .yml 
#por lo que no es necesario modificar nada, este paso se puede dar RUN.
#Verifica que el kernel termine de cargar, no accedas al siguiente paso hasta que este no cargue por completo
# ya que tu equipo se sobrecargara de tareas y cargará el codigo mas lento.

#  pip install --upgrade Pillow

import cv2
import os
import numpy as np
from PIL import Image
import pickle

cascPath = "./Cascades/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

#reconocimiento con opencv
reconocimiento = cv2.face.LBPHFaceRecognizer_create()

BASE_DIR = os.path.dirname(os.path.abspath('__file__')) 
image_dir = os.path.join(BASE_DIR,"images")


current_id = 0
etiquetas_id = {}
y_etiquetas = []
x_entrenamiento = []

for root, dirs, archivos in os.walk(image_dir):
    for archivo in archivos:
        if archivo.endswith("png") or archivo.endswith("jpg"):
            pathImagen = os.path.join(root,archivo)
            etiqueta = os.path.basename(root).replace(" ", "-")#.lower()
            #print(etiqueta,pathImagen)

            #Creando las etiquetas
            if not etiqueta in etiquetas_id:                
                etiquetas_id[etiqueta] = current_id
                current_id += 1            
            id_ = etiquetas_id[etiqueta]
            #print(etiquetas_id)

            pil_image = Image.open(pathImagen).convert("L")
            tamanio = (550,550)
            imagenFinal = pil_image.resize(tamanio, Image.ANTIALIAS)
            image_array = np.array(pil_image,"uint8")
            #print(image_array)

            rostros = faceCascade.detectMultiScale(image_array, 1.5, 5)

            for (x,y,w,h) in rostros:
                roi = image_array[y:y+h, x:x+w]
                x_entrenamiento.append(roi)
                y_etiquetas.append(id_)


#print(y_etiquetas)                
#print(x_entrenamiento)
with open("labels.pickle",'wb') as f:
    pickle.dump(etiquetas_id, f)

reconocimiento.train(x_entrenamiento, np.array(y_etiquetas))
reconocimiento.save("entrenamiento.yml")

C:\Users\deiby\AppData\Local\Temp\ipykernel_5708\260978897.py:45: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  imagenFinal = pil_image.resize(tamanio, Image.ANTIALIAS)


In [ ]:
#reconocimiento

In [40]:
# en esta parte solo es necesario modificar la etiqueta con el nombre de la persona que se registró 

import cv2
import pickle

cascPath = "./Cascades/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

eyeCascade = cv2.CascadeClassifier("./Cascades/haarcascade_eye.xml")
smileCascade = cv2.CascadeClassifier("./Cascades/haarcascade_smile.xml")

reconocimiento = cv2.face.LBPHFaceRecognizer_create()
reconocimiento.read("./entrenamiento.yml")

etiquetas = {"deiby" : 1 }    #esta parte es la que modificaremos con el nombre de la persona que se registro 

with open("labels.pickle",'rb') as f:
    pre_etiquetas = pickle.load(f)
    etiquetas = { v:k for k,v in pre_etiquetas.items()}

web_cam = cv2.VideoCapture(0)

while True:
    # Capture el marco
    ret, marco = web_cam.read()
    grises = cv2.cvtColor(marco, cv2.COLOR_BGR2GRAY)    
    rostros = faceCascade.detectMultiScale(grises, 1.5, 5)

    # Dibujar un rectángulo alrededor de las rostros
    for (x, y, w, h) in rostros:
        #print(x,y,w,h)
        roi_gray = grises[y:y+h, x:x+w]
        roi_color = marco[y:y+h, x:x+w]

        # reconocimiento
        id_, conf = reconocimiento.predict(roi_gray)
        if conf >= 4  and conf < 85:                
            #print(id_)
            #print(etiquetas[id_])           
            font = cv2.FONT_HERSHEY_SIMPLEX            

            nombre = etiquetas[id_]

            if conf > 50:
                #print(conf)
                nombre = "..."   #esta parte nos dice que si no se reconoce el rostro coloca como "desconocido" hasta que detecte rasgos 

            color = (255,255,255)
            grosor = 2
            cv2.putText(marco, nombre, (x,y), font, 1, color, grosor, cv2.LINE_AA)

        img_item = "my-image.png" 
        cv2.imwrite(img_item, roi_gray)
        
        cv2.rectangle(marco, (x, y), (x+w, y+h), (0, 255, 0), 2)

        rasgos = smileCascade.detectMultiScale(roi_gray)
        for(ex,ey,ew,eh) in rasgos:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
    
    # Display resize del marco  
    marco_display = cv2.resize(marco, (1200, 650), interpolation = cv2.INTER_CUBIC)
    cv2.imshow('Detectando Rostros', marco_display)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cuando todo está hecho, liberamos la captura
web_cam.release()
cv2.destroyAllWindows()


#luego de leer todas las notas descritas en el programa le damos a RUN y hacemos la misma parte 
# en caso que prenda el led y no muestre la ventana emergente ubicamos bien el rostro para que detecte una cara 
#si queremos salirnos de la pantalla emergente le damos a la letra " q "

#SI SE QUEDA ENCENDIDO EL LED DESPUES QUE LE DIMOS A "q" PARA SALIR, REINICIAR EL KERNEL Y LISTO 